In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from rich import print
from tqdm import tqdm

In [2]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """ Convert a list of string to a list of other types
    
    pattern: a list of types for one item. 
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [3]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

head = convert_str_li_to_other_li(
    lines, pattern="i", per_letter=False, sep=",", start_row=0, end_row=1
)

print(f"Head:\n{head}")

data = convert_str_li_to_other_li(
    lines, pattern="i", per_letter=False, sep=" ", start_row=2, end_row=None
)
# data = convert_str_li_to_other_li(
#     lines, pattern="si", per_letter=False, sep=" ", start_row=0, end_row=None
# )
# data = convert_str_li_to_other_li(
#     lines, pattern="i", per_letter=True, sep=" ", start_row=0, end_row=None
# )

print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: 1,76,38,96,62,41,27,33,4,2,94,15,89,25,66,14,30,0,71,21,48,44,87,73,60,50,77,45,2
9,18,5,99,65,16,93,95,37,3,52,32,46,80,98,63,92,24,35,55,12,81,51,17,70,78,61,91,54,8,72,40,7
4,68,75,67,39,64,10,53,9,31,6,7,47,42,90,20,19,36,22,43,58,28,79,86,57,49,83,84,97,11,85,26,6
9,23,59,82,88,34,56,13 | Last line: 46 50 48 76  5

------

Convert to:

Head:
[[1, 76, 38, 96, 62, 41, 27, 33, 4, 2, 94, 15, 89, 25, 66, 14, 30, 0, 71, 21, 48, 44, 87, 73,
60, 50, 77, 45, 29, 18, 5, 99, 65, 16, 93, 95, 37, 3, 52, 32, 46, 80, 98, 63, 92, 24, 35, 55,
12, 81, 51, 17, 70, 78, 61, 91, 54, 8, 72, 40, 74, 68, 75, 67, 39, 64, 10, 53, 9, 31, 6, 7, 
47, 42, 90, 20, 19, 36, 22, 43, 58, 28, 79, 86, 57, 49, 83, 84, 97, 11, 85, 26, 69, 23, 59, 
82, 88, 34, 56, 13]]

First line: [85, 23, 65, 78, 93]

Last line: [46, 50, 48, 76, 5]

------

In [4]:
class Bingo_Board_Set:
    def __init__(self, set_numbers):
        self.numbers = np.array(set_numbers)
        self.mask = np.full_like(set_numbers, False)
        self.shape = self.numbers.shape

    def set_a_new_num(self, new_num):
        self.mask |= self.numbers == new_num

    def set_a_list_num(self, num_li):
        for num in num_li:
            self.set_a_new_num(num)

    def print_current_mask(self):
        print(self.mask)

    def print_numbers(self):
        print(self.numbers)

    def check_win(self):
        win = False
        for idx, total_count in enumerate(self.shape):
            win |= np.any(np.sum(self.mask, axis=idx) == total_count)
        return win

    def count_masked(self):
        return np.sum(self.numbers * self.mask)

    def count_unmasked(self):
        return np.sum(self.numbers) - self.count_masked()


def load_data(data):
    all_set = []
    total_set = (len(data) + 1) // 6
    for set_num in range(total_set):
        all_set.append(Bingo_Board_Set(data[set_num * 6 : set_num * 6 + 5]))
    #     print("Last set:")
    #     all_set[total_set-1].print_numbers()
    return all_set


all_set = load_data(data)


def play_game(head, all_set):
    win_or_not = np.zeros(len(all_set))
    all_res = []
    for num in head[0]:
        for idx, one_set in enumerate(all_set):
            if win_or_not[idx] == 0:
                one_set.set_a_new_num(num)
                if one_set.check_win():
                    win_or_not[idx] = 1
                    res = one_set.count_unmasked() * num
                    all_res.append(res)
    return all_res


all_res = play_game(head, all_set)
print(f"Answer to Q1: {all_res[0]}")
print(f"Answer to Q2: {all_res[-1]}")

Answer to Q1: 5685

Answer to Q2: 21070